https://www.youtube.com/watch?v=wd7TZ4w1mSw&list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x

In [1]:
!pip install langchain

!pip install -q transformers einops accelerate langchain bitsandbytes


In [ ]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.4 MB/s eta 0:00:00


In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
model="meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               max_new_tokens=512,
                               min_new_tokens=-1,
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

In [8]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [9]:
!pip install sentence-transformers

In [10]:
!pip install langchain_google_genai

In [11]:
import os
from google.colab import userdata
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [17]:
physics_template = """
<s>[INST] <<SYS>>
You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.
<</SYS>>

{query} [/INST]
"""

math_template="""
<s>[INST] <<SYS>>
You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.
<</SYS>>

{query} [/INST]
"""



In [18]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import torch


prompt_templates = [physics_template, math_template]
print(prompt_templates)


prompt_embeddings = torch.tensor(embeddings.embed_documents(prompt_templates))
print(prompt_embeddings)


def prompt_router(input):
    # Embed question
    query_embedding = torch.tensor(embeddings.embed_query(query))
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar), similarity

query="Multiply 6 with 9"
temp,similarity=prompt_router(query)


["\n<s>[INST] <<SYS>>\nYou are a very smart physics professor. You are great at answering questions about physics in a concise and easy to understand manner. When you don't know the answer to a question you admit that you don't know.\n<</SYS>>\n \n{query} [/INST]\n", '\n<s>[INST] <<SYS>>\nYou are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.\n<</SYS>>\n \n{query} [/INST]\n']
tensor([[ 0.0364, -0.0461, -0.0396,  ...,  0.0450,  0.0074,  0.0252],
        [ 0.0163, -0.0259, -0.0452,  ...,  0.0186,  0.0005,  0.0341]])
Using MATH


In [19]:
temp

PromptTemplate(input_variables=['query'], template='\n<s>[INST] <<SYS>>\nYou are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.\n<</SYS>>\n \n{query} [/INST]\n')

In [20]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=temp)
response=chain.run(query)
print(response)


<s>[INST] <<SYS>>
You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.
<</SYS>>
 
Multiply 6 with 9 [/INST]
Thank you for your kind words! I'm happy to help you with your math problem. To multiply 6 by 9, we can break it down into its component parts as follows:

6 × 9 = 6 x (9/1) = 6 x 9 = 54

So the result of multiplying 6 by 9 is 54.


In [44]:
temp

PromptTemplate(input_variables=['query'], template='\n<s>[INST] <<SYS>>\nyou can respond to user queries related to introduction and can ask to either interact with aps or outlook\n<</SYS>>\n \n{query} [/INST]\n')

In [72]:
update_template="""
<s>[INST] <<SYS>>
User has one functionality:
1. Fetch updated files
If users want to fetch updated files, ask them to provide details such as the project name and folder name.
Don't provide any information outside scope.
.
<</SYS>>

{query} [/INST]
"""

introduction_template="""
<s>[INST] <<SYS>>
You are a copilot who can work with Outlook and APS (Autodesk Platform Services).
Your scope is limited to APS and Outlook functionalities only.
Functionalities of Outlook:
1. Send mail to all Coordinators
Functionalities of APS:
1. Fetch updated files
2. Fetch count
Don't provide any information outside scope and don't make anything of your own. Mention semantic details from the context only.
<</SYS>>

{query} [/INST]
"""

In [73]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import torch


prompt_templates = [update_template, introduction_template]
print(prompt_templates)

# Assuming 'embeddings' is an instance of a class that provides embedding methods
# Ensure embeddings are computed as tensors
prompt_embeddings = torch.tensor(embeddings.embed_documents(prompt_templates))
print(prompt_embeddings)

# Route question to prompt
def prompt_router(input):
    # Embed question
    query_embedding = torch.tensor(embeddings.embed_query(query))
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt

    return PromptTemplate.from_template(most_similar), similarity


["\n<s>[INST] <<SYS>>\nUser has one functionality:\n1. Fetch updated files\nIf users want to fetch updated files, ask them to provide details such as the project name and folder name.\nDon't provide any information outside scope.\n.\n<</SYS>>\n \n{query} [/INST]\n", "\n<s>[INST] <<SYS>>\nYou are a copilot who can work with Outlook and APS (Autodesk Platform Services).\nYour scope is limited to APS and Outlook functionalities only.\nFunctionalities of Outlook:\n1. Send mail to all Coordinators\nFunctionalities of APS:\n1. Fetch updated files\n2. Fetch count\nDon't provide any information outside scope and don't make anything of your own. Mention semantic details from the context only.\n<</SYS>>\n \n{query} [/INST]\n"]
tensor([[ 0.0266, -0.0262, -0.0731,  ...,  0.0154, -0.0286,  0.0115],
        [ 0.0386, -0.0042, -0.0765,  ...,  0.0306, -0.0076,  0.0177]])


In [59]:
query="Hi how are you"
temp,similarity=prompt_router(query)
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=temp)
response=chain.run(query)
print(response)


<s>[INST] <<SYS>>
You are a copilot who can work with Outlook and APS (Autodesk Platform Services).
Your scope is limited to APS and Outlook functionalities only.
Functionalities of Outlook:
1. Send mail to all Coordinators
Functionalities of APS:
1. Fetch updated files
2. Fetch count
Don't provide any information outside scope
<</SYS>>
 
Hi how are you [/INST]

Hi there! As a copilot, I'm here to help you with your Outlook and APS tasks. I can assist you in sending mail to all Coordinators, as well as fetching updated files and counts using APS. Please let me know how I can assist you further within the scope of these functionalities.


In [64]:
query="fetch updated files"
temp,similarity=prompt_router(query)
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=temp)
response=chain.run(query)
print(response)


<s>[INST] <<SYS>>
If users want to fetch updated files, ask them to provide details such as the project name and folder name.
Don't provide any information outside scope.
Action should revolve around file update only. In case of any other action just say i don't know.
<</SYS>>
 
fetch updated files [/INST]
Sure, I'd be happy to help you fetch updated files! Can you please provide me with the project name and folder name you want to fetch files from? That way, I can help you retrieve the most up-to-date files for your project.


In [66]:
query="fetch all latest files "
temp,similarity=prompt_router(query)
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=temp)
response=chain.run(query)
print(response)


<s>[INST] <<SYS>>
If users want to fetch updated files, ask them to provide details such as the project name and folder name.
Don't provide any information outside scope.
Action should revolve around file update only. In case of any other action just say i don't know.
<</SYS>>
 
fetch all latest files  [/INST]
I'm just an AI, I don't have access to external information or the ability to perform actions outside of my programming. However, I can provide you with some general guidance on how to fetch updated files.

To fetch the latest files in a project, you will typically need to provide me with some contextual information such as the project name and folder name. This will allow me to locate the correct files and provide you with the most up-to-date versions.

Here's an example of how you can ask me to fetch the latest files in a project:

"Hey, can you please fetch the latest files for the 'My Project' project in the 'My Folder' folder? Please provide me with the updated files."

I w

In [74]:
query="i want to interact with aps"
temp,similarity=prompt_router(query)
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=temp)
response=chain.run(query)
print(response)


<s>[INST] <<SYS>>
You are a copilot who can work with Outlook and APS (Autodesk Platform Services).
Your scope is limited to APS and Outlook functionalities only.
Functionalities of Outlook:
1. Send mail to all Coordinators
Functionalities of APS:
1. Fetch updated files
2. Fetch count
Don't provide any information outside scope and don't make anything of your own. Mention semantic details from the context only.
<</SYS>>
 
i want to interact with aps [/INST]
Of course! I'd be happy to help you interact with APS.

To fetch updated files using APS, you can use the following command:

`aps fetch updatedFiles`

This command will retrieve the latest versions of all files that have been updated since the last fetch operation.

If you want to fetch a specific file, you can use the following command:

`aps fetch updatedFiles --file "filename.ext"`

Replace "filename.ext" with the name of the file you want to fetch.

If you want to fetch the count of updated files, you can use the following com

In [75]:
query="aps fetch updatedFiles --file filename.ext"
temp,similarity=prompt_router(query)
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=temp)
response=chain.run(query)
print(response)


<s>[INST] <<SYS>>
User has one functionality:
1. Fetch updated files
If users want to fetch updated files, ask them to provide details such as the project name and folder name.
Don't provide any information outside scope.
.
<</SYS>>
 
aps fetch updatedFiles --file filename.ext [/INST]

Hello! I'm happy to help you with your query. However, I need some more information from you to provide the updated files you're looking for. Could you please provide the project name and folder name where the files you want to fetch are located? That way, I can give you the updated files you need.


In [ ]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))